In [1]:
# imports
import pandas as pd
import matplotlib.pyplot

In [2]:
# read files 
private_data_original = pd.read_csv('../data/private_dataB.csv')
public_data_register = pd.read_csv('../data/public_data_registerB.csv')
public_data_results = pd.read_csv('../data/public_data_resultsB.csv')

## Doin transforms you know they did too